In [ ]:
#!pip install crewai crewai_tools langchain==0.2.6 langchain-core==0.2.15 langchain-experimental==0.0.62 langchain-community==0.2.6 langchain_mistralai

In [1]:
from dotenv import load_dotenv
import os
local=True
load_dotenv(".env_generic")
load_dotenv(".env_neo4j_local")
model_file=".env_mistal_ai"
if (local):
    model_file=".env_ollama"
    model_file=".env_codestral" 
#    model_file=".env_openai" 
load_dotenv(model_file)
print(os.getenv('OPENAI_MODEL_NAME'),os.getenv('OPENAI_API_BASE'))

codestral-latest https://codestral.mistral.ai/v1/


In [2]:
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import Tool
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
import os
graph = Neo4jGraph(url=os.environ['NEO4J_URI'], username=os.environ['NEO4J_USERNAME'], password=os.environ['NEO4J_PASSWORD'])
graph.refresh_schema()
llm = ChatOpenAI(model=os.getenv('OPENAI_MODEL_NAME'), temperature=0)


CYPHER_GENERATION_TEMPLATE = """
Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
All the data is based on ifc (industry foundation classes) standard.
Do not use any other relationship types or properties that are not provided.
when you get a question that is cypher, dont execute it as is, try to understand what the question is about and translate it according to the knowledge that you have, as you are top expert of getting the data, and the one that question is not aware of the data model.
Schema:
{schema}
Cypher examples:
# get gross volume of all entities that has gross volume (such as IfcBeam, IfcWallStandardCase, IfcSlab, IfcColumn): 
             MATCH (n:IfcQuantityVolume)<-[:Quantities]-(:IfcElementQuantity)<-[:RelatingPropertyDefinition]-(:IfcRelDefinesByProperties)-[:RelatedObjects]->(o) return labels(o)[0] as entity_name,o.ObjectType as entity_type ,n.VolumeValue as gross_volume
# get all columns with relation to floors:
            MATCH (n:IfcBuildingStorey)<-[:RelatingStructure]-(ss:IfcRelContainedInSpatialStructure)-[:RelatedElements]->(c:IfcColumn) RETURN n,ss,c
# whar are the materials used for beam?
             match (mt:IfcMaterial)<-[:RelatingMaterial]-(ascmat:IfcRelAssociatesMaterial)-[:RelatedObjects]->(bm:IfcBeam ) return (mt) .
# what objects were made by materials ?
            MATCH (m:IfcMaterial)<-[:RelatingMaterial]-(asm:IfcRelAssociatesMaterial)-[:RelatedObjects]->(o) return (o)
# what spaces spaces are in floors?
            match(IfcBuildingStorey)<-[:RelatingObject]-(:IfcRelAggregates)-[:RelatedObjects]->(s:IfcSpace) return (s)
# count beams by type:
            MATCH (b:IfcBeam) RETURN b.ObjectType AS object_type, COUNT(b) AS count ORDER BY count DESC
# count columns by type:
            MATCH (b:IfcColumns) RETURN b.ObjectType AS object_type, COUNT(b) AS count ORDER BY count DESC
# count walls by type:
            MATCH (b:IfcWall) RETURN b.ObjectType AS object_type, COUNT(b) AS count ORDER BY count DESC
# get the buildings floor/storey sequence, Storey Level, Storey Elevation and Storey Name:
            MATCH (storey:IfcBuildingStorey) WHERE storey.Elevation IS NOT NULL WITH storey ORDER BY storey.Elevation ASC WITH collect(storey) AS sortedStoreys UNWIND range(0, size(sortedStoreys) - 1) AS idx WITH idx, sortedStoreys[idx] AS storey, sortedStoreys RETURN CASE WHEN storey.Elevation < 0 THEN -1 * (size([s IN sortedStoreys WHERE s.Elevation < 0]) - idx) WHEN storey.Elevation = 0 THEN 0 ELSE idx - size([s IN sortedStoreys WHERE s.Elevation < 0]) END AS StoreyLevel, idx+1 as StoreySequence,storey.Name AS StoreyName, storey.Elevation AS Elevation ORDER BY storey.Elevation ASC

# what are the relation between IfcBuilding and  IfcBuildingStorey storeys:
            MATCH (b:IfcBuilding)<-[:RelatingObject]-(:IfcRelAggregates)-[:RelatedObjects]->(bs:IfcBuildingStorey) return(bs)
# get the building top floor/storey name:
            MATCH (storey:IfcBuildingStorey) WHERE storey.Elevation IS NOT NULL WITH storey ORDER BY storey.Elevation DESC LIMIT 1 RETURN storey.Name AS TopStoreyName
# get the  storey level of the middle storey:
            MATCH (storey:IfcBuildingStorey) WHERE storey.Elevation IS NOT NULL WITH storey ORDER BY storey.Elevation ASC WITH collect(storey) AS sortedStoreys WITH sortedStoreys, size(sortedStoreys) AS totalCount WITH sortedStoreys, totalCount, floor(totalCount / 2.0) AS middleIndex RETURN sortedStoreys[toInteger(middleIndex)] AS middleStorey, sortedStoreys[toInteger(middleIndex)].Name AS StoreyName, sortedStoreys[toInteger(middleIndex)].Elevation AS Elevation

# get the number of columns, sum of column volume, sum of beam volume,  average of beam volume,  average of column volume for each storey of the building, mention the storey name and elevation order by elevation:
            MATCH (storey:IfcBuildingStorey)<-[:RelatingStructure]-(ss:IfcRelContainedInSpatialStructure)-[:RelatedElements]->(c:IfcColumn) OPTIONAL MATCH (c)<-[:RelatedObjects]-(eq:IfcRelDefinesByProperties)-[:RelatingPropertyDefinition]->(ev:IfcElementQuantity)-[:Quantities]->(qv:IfcQuantityVolume) WITH storey, count(c) AS column_count, sum(qv.VolumeValue) AS column_volume_sum, storey.Name AS storey_name, storey.Elevation AS storey_elevation OPTIONAL MATCH (storey)<-[:RelatingStructure]-()-[:RelatedElements]->(b:IfcBeam) OPTIONAL MATCH (b)<-[:RelatedObjects]-()-[:RelatingPropertyDefinition]->(ev:IfcElementQuantity)-[:Quantities]->(qv:IfcQuantityVolume) WITH storey, column_count, column_volume_sum, storey_name, storey_elevation, count(b) AS beam_count, sum(qv.VolumeValue) AS beam_volume_sum RETURN storey_name, storey_elevation, column_count, column_volume_sum, beam_count, beam_volume_sum, CASE WHEN beam_count > 0 THEN beam_volume_sum / beam_count ELSE 0 END AS beam_volume_avg, CASE WHEN column_count > 0 THEN column_volume_sum / column_count ELSE 0 END AS column_volume_avg ORDER BY storey_elevation ASC
Critical Note: 
Do not include any explanations or apologies in your responses.
Do not respcrond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement. 
Do Not respond with any additional text except for the cypher itself, as it causes critical failures


The question is:
{question}"""
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

neo4jquery = GraphCypherQAChain.from_llm(
    llm, graph=graph, top_k=200,verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT
)
neo4j_query_tool = Tool(
  name="neo4j_query_tool",
  func=neo4jquery.run,
  description="provide information about building project. the information is stored according to IFC standard. The tool is familiar with the datamodel and can provide raw data as well as aggregared data (such as total volume by floor, number of elements, etc..), user provide english question the tool translates it to query and respond with the answer",
  cypher_prompt=CYPHER_GENERATION_PROMPT
)
#neo4j_query_tool.run("get the number of columns, sum of column volume, sum of beam volume,  average of beam volume,  average of column volume for each storey of the building, mention the storey name and elevation order by elevation ")

In [3]:
# Agent 1: Researcher
from crewai import Agent, Task, Crew
analyst = Agent(
    role="civil engineer",
    goal="provide information about building materials, quantities and sizes, be as accurate as possible",
    tools = [  neo4j_query_tool],
    verbose=True,
    backstory=(
        "As civil engineer, you get your input from the neo4j_query_tool, you ask the questions in human language, for example, if you want to know number of floors you ask : how many floors are there? and not a cypher that you think that is right "
        "as the tool translates the query better than you. "
        "The tool has arythmetic capabilities, and you should use it as much as possible. "
        "The tool has information that is pre calculated, so you dont need to perform the calculations for things, like volumes. "
        "usually the tool should respond to the questions, including aggregation and sums. only if you are not satisfied from the answer, you might need to calculate things by yourself, according to the data you get from the tool. you should not invent anything!!! "
        "sometimes you need to conclude things about entities from their name or ObjectType. and then you should get the value and conclude by yourself. "
        "If you would need to conclude something from column name by yourself you should do it, the default is not to do so. "
        "for example, if you are requested to count beams by type and for each type get the properties, such as: material, shape or dimensions, you shold follow the following steps: "
        "1) request from tool: count number of beams by ObjectType. "
        "2) using the information you got from the tool, look at the value of ObjectType, and conclude fron the text what are the properties "
        "3) only if you dont find the requested properties, try to get it be other relationships. "
        "getting data for walls and columns, should follow  similiar process as of the beam example."
    )
)
# Task for Researcher Agent:
analyst_task = Task(
    description=(
#        "get the columns and their types, for each type, calculate how many instances. the ObjectType property each wall type is a description of the column. identify the material,  Shape,  Dimensions"
        "provide number and volume of beams by floor"
    ),
    expected_output=(
#        "json with the information of types with their properties from the text of the type. count how many of each type. "
#        "do not write enything else except for this json"
        "json with the result by floor"
    ),
    agent=analyst,
    async_execution=True
)
application_crew = Crew(
    agents=[analyst],

    tasks=[analyst_task],

    verbose=True
)
result = application_crew.kickoff()


 [2024-08-09 14:35:51][DEBUG]: == Working Agent: civil engineer
 [2024-08-09 14:35:51][INFO]: == Starting Task: provide number and volume of beams by floor


> Entering new CrewAgentExecutor chain...
I need to find out the number and volume of beams by floor. I'll start by counting the number of beams by floor using the neo4j_query_tool.

Action: neo4j_query_tool
Action Input: {"tool_input": "count number of beams by floor"}

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (b:IfcBeam)<-[:RelatedElements]-(ss:IfcRelContainedInSpatialStructure)-[:RelatingStructure]->(storey:IfcBuildingStorey)
RETURN storey.Name AS storey_name, storey.Elevation AS storey_elevation, COUNT(b) AS beam_count
ORDER BY storey_elevation ASC
Full Context:
[{'storey_name': 'גלריה 0', 'storey_elevation': 299.99999999998664, 'beam_count': 6}, {'storey_name': 'קומה 1', 'storey_elevation': 564.9999999999869, 'beam_count': 94}, {'storey_name': 'גלריה 1', 'storey_elevation': 864.9999999999874, 'beam_c

In [ ]:
import json
result.